In [1]:
import os
import urllib2
from BeautifulSoup import BeautifulSoup

In [2]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [34]:
base_url = 'https://www.rt.com/listing/last-news.category.{}/prepare/default/200/0/'
categories = ['uk', 'politics', 'usa', 'business', 'sport']

site_urls = {}

for category in log_progress(categories):
    tmpurls = []
    
    page = urllib2.urlopen(base_url.format(category))
    soup = BeautifulSoup(page)

    articles = soup.findAll('strong')
    print '{}: {}'.format(category, len(articles))

    for article in articles:
        title = article.contents[0]
        if title.attrs[1][0] == 'href':
            tmpurl = 'https://www.rt.com' + title.attrs[1][1]
            tmpurls.append(tmpurl)
    site_urls[category] = tmpurls
#     break

uk: 200
politics: 200
usa: 200
business: 200
sport: 200


In [36]:
class text_unit(object):
    
    def __init__(self, cat=None, title=None, text=None):
        self.category = cat
        self.title = title
        self.text = text

class corpus(object):
    
    def __init__(self, ent=[]):
        self.entities = []
        
    def add(self, entity):
        self.entities.append(entity)
        
    @property
    def length(self):
        return len(self.entities)

In [42]:
corp = corpus()

for cat, urls in site_urls.iteritems():
    for url in log_progress(urls):
        page = urllib2.urlopen(url)
        soup = BeautifulSoup(page)

        soup_title = soup.findAll('h1', attrs={'class': 'article__heading'})
        title = str(soup_title[0].contents[0]).strip()

        soup_text = soup.findAll('div', attrs={'class': 'article__text text '})
        text = ''
        for el in soup_text[0].contents:
            if el.name == 'p':
                text += el.text + '\n'
        #     break
        
        tu = text_unit(cat, title, text)
        corp.add(tu)
#     break

In [43]:
print corp.length

1000
